## Attempt 2

This is a second and cleaner attempt. The main difference this time is that instead of just dropping the NA columns I use simple imputation with mean (could have used median, but it's a longer word to type). You do not have to run the first two code blocks - I have uploaded the imputed .csv files for the train and test data sets separately. 

Steps in this code:
1. Load the data
2. Imputation
3. Create the model
4. Train the model on the training data
5. Predict and make .csv
6. Win lots and lots of money

Things that could probably make this better:
- Dimensionality reduction on the fMRI data (needs scaling)
- Iteratively work through the ann options to find the best model (can def code this and make it automatic, not like me manually messing with the numbers)
    - The code I'm messing with at the bottom involves this bit to see if I can improve the bad f1 score I got with default values 

In [2]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer

# Step 1: Load the training and test data (update paths as necessary)
train_fmri_path = 'widsdatathon2025/TRAIN/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv'  # Replace with actual file path
train_quant_path = 'widsdatathon2025/TRAIN/TRAIN_QUANTITATIVE_METADATA.xlsx'  # Replace with actual file path
train_cat_path = 'widsdatathon2025/TRAIN/TRAIN_CATEGORICAL_METADATA.xlsx'  # Replace with actual file path

test_fmri_path = 'widsdatathon2025/TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv'  # Replace with actual file path
test_quant_path = 'widsdatathon2025/TEST/TEST_QUANTITATIVE_METADATA.xlsx'  # Replace with actual file path
test_cat_path = 'widsdatathon2025/TEST/TEST_CATEGORICAL.xlsx'  # Replace with actual file path

# Load the datasets
train_fmri_data = pd.read_csv(train_fmri_path)
train_quant_data = pd.read_excel(train_quant_path)
train_cat_data = pd.read_excel(train_cat_path)

test_fmri_data = pd.read_csv(test_fmri_path)
test_quant_data = pd.read_excel(test_quant_path)
test_cat_data = pd.read_excel(test_cat_path)

# Step 2: Merge the training data based on 'participant_id' using an inner join
train_data = pd.concat([train_fmri_data, train_quant_data, train_cat_data], axis=1)
train_data_merged = pd.merge(train_fmri_data, train_quant_data, on='participant_id', how='inner')
train_data_merged = pd.merge(train_data_merged, train_cat_data, on='participant_id', how='inner')

# Step 3: Merge the test data based on 'participant_id' using an inner join
test_data = pd.concat([test_fmri_data, test_quant_data, test_cat_data], axis=1)
test_data_merged = pd.merge(test_fmri_data, test_quant_data, on='participant_id', how='inner')
test_data_merged = pd.merge(test_data_merged, test_cat_data, on='participant_id', how='inner')


# Use SimpleImputer for a faster alternative
imputer = SimpleImputer(strategy='mean')

# Imputation for training data (excluding participant_id)
train_data_imputed = imputer.fit_transform(train_data_merged.drop(columns=['participant_id']))
train_data_imputed_df = pd.DataFrame(train_data_imputed, columns=train_data_merged.columns[1:])
train_data_imputed_df['participant_id'] = train_data_merged['participant_id']  # Add back the participant_id column

# Imputation for test data (excluding participant_id)
test_data_imputed = imputer.fit_transform(test_data_merged.drop(columns=['participant_id']))
test_data_imputed_df = pd.DataFrame(test_data_imputed, columns=test_data_merged.columns[1:])
test_data_imputed_df['participant_id'] = test_data_merged['participant_id']  # Add back the participant_id column

# Step 5: Check the imputed training and test data
print("Imputed Training Data:")
print(train_data_imputed_df.head())

print("Imputed Test Data:")
print(test_data_imputed_df.head())

# Now you have imputed training and test datasets ready for further processing


Imputed Training Data:
   0throw_1thcolumn  0throw_2thcolumn  0throw_3thcolumn  0throw_4thcolumn  \
0          0.093473          0.146902          0.067893          0.015141   
1          0.029580          0.179323          0.112933          0.038291   
2         -0.051580          0.139734          0.068295          0.046991   
3          0.016273          0.204702          0.115980          0.043103   
4          0.065771          0.098714          0.097604          0.112988   

   0throw_5thcolumn  0throw_6thcolumn  0throw_7thcolumn  0throw_8thcolumn  \
0          0.070221          0.063997          0.055382         -0.035335   
1          0.104899          0.064250          0.008488          0.077505   
2          0.111085          0.026978          0.151377          0.021198   
3          0.056431          0.057615          0.055773          0.075030   
4          0.071139          0.085607          0.019392         -0.036403   

   0throw_9thcolumn  0throw_10thcolumn  ...  Basic_

In [3]:
# Step 1: Save the training data (assuming `full_train_data` is your processed training data)
train_data_file_path = 'processed_train_data.csv'  # Specify the desired file path
train_data_imputed_df.to_csv(train_data_file_path, index=False)  # Save without index column

print(f"Training data saved to {train_data_file_path}")

# Step 2: Save the test data (assuming `test_full_data` is your processed test data)
test_data_file_path = 'processed_test_data.csv'  # Specify the desired file path
test_data_imputed_df.to_csv(test_data_file_path, index=False)  # Save without index column

print(f"Test data saved to {test_data_file_path}")


Training data saved to processed_train_data.csv
Test data saved to processed_test_data.csv


Run this block of code to load in the .csv files of the imputed and merged data and go from here 

In [ ]:
import pandas as pd
# Load in the training data
train_data_imputed_df = pd.read_csv('processed_train_data.csv')
test_data_imputed_df = pd.read_csv('processed_test_data.csv')

#### Training the Model

In [6]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


# --- Custom Dataset for Multi-Task Learning ---
class MultiTaskDataset(Dataset):
    def __init__(self, df, target_cols):
        self.y = df[target_cols].values.astype(np.float32)
        self.X = df.drop(columns=target_cols + ['participant_id']).values.astype(np.float32)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]  # y is a vector of targets

# --- Custom Weighted Loss Function ---
class WeightedBCELoss(nn.Module):
    def __init__(self):
        super(WeightedBCELoss, self).__init__()

    def forward(self, preds, target, sex, adhd):
        # Assign a weight of 2 for female ADHD cases (Sex_F=1, ADHD_Outcome=1)
        weights = torch.ones_like(target[:, 0])  # Default weight is 1, shape (batch_size,)
        weights[(sex == 1) & (adhd == 1)] = 2  # Apply weight of 2 for female ADHD cases
        
        # Compute the loss for each task
        loss_sex = nn.BCELoss(reduction='none')(preds[:, 0], target[:, 0])  # Loss for Sex_F
        loss_adhd = nn.BCELoss(reduction='none')(preds[:, 1], target[:, 1])  # Loss for ADHD_Outcome
        
        # Apply the weights to each task's loss
        weighted_loss_sex = loss_sex * weights  # Apply weight for Sex_F loss
        weighted_loss_adhd = loss_adhd * weights  # Apply weight for ADHD_Outcome loss

        return (weighted_loss_sex.mean() + weighted_loss_adhd.mean()) / 2

# --- Multi-Task Neural Network ---
class MultiTaskNN(nn.Module):
    def __init__(self, input_dim, dropout_rate=0.3):
        super(MultiTaskNN, self).__init__()
        # Shared layers with dropout
        self.shared = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(dropout_rate),  # Dropout added here
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(dropout_rate)  # Dropout added here
        )
        # Separate heads for each task
        self.sex_head = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
        self.adhd_head = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        shared_rep = self.shared(x)
        sex_pred = self.sex_head(shared_rep)
        adhd_pred = self.adhd_head(shared_rep)
        return torch.cat([sex_pred, adhd_pred], dim=1)  # output size [batch_size, 2]

# --- F1 Score Calculation with Weights ---
def weighted_f1_score(preds, target, sex, adhd):
    weights = np.ones_like(target[:, 0])  # Default weight is 1 for both tasks
    weights[(sex == 1) & (adhd == 1)] = 2  # Apply weight of 2 for female ADHD cases
    
    # Calculate F1 score for both tasks
    f1_sex = f1_score(target[:, 0], (preds[:, 0] > 0.5).astype(int), sample_weight=weights)
    f1_adhd = f1_score(target[:, 1], (preds[:, 1] > 0.5).astype(int), sample_weight=weights)
    
    return (f1_sex + f1_adhd) / 2

# --- Training Loop ---
def train_model(df, target_cols, num_epochs=100, batch_size=64, learning_rate=0.0001, weight_decay=0.01, dropout_rate=0.2):
    # Set device (this should be defined before using it)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Split the data into training and validation sets (80-20 split)
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
    
    # Create datasets and dataloaders
    train_dataset = MultiTaskDataset(train_df, target_cols)
    val_dataset = MultiTaskDataset(val_df, target_cols)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Initialize the model
    input_dim = train_df.drop(columns=target_cols + ['participant_id']).shape[1]
    model = MultiTaskNN(input_dim, dropout_rate).to(device)
    
    # Initialize custom weighted loss
    criterion = WeightedBCELoss()

    # Define optimizer with weight decay (L2 regularization)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    
    best_val_f1 = 0  # For tracking best F1 score
    model_weights_per_epoch = []  # Store model weights at each epoch
    val_f1_scores = []  # Store F1 scores for validation

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        train_losses = []
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            sex = y_batch[:, 0]  # Extract Sex_F values
            adhd = y_batch[:, 1]  # Extract ADHD_Outcome values
            
            optimizer.zero_grad()
            preds = model(X_batch)
            
            # Calculate weighted loss
            loss = criterion(preds, y_batch, sex, adhd)
            
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
        
        avg_train_loss = np.mean(train_losses)
        
        # --- Validation ---
        model.eval()
        all_preds = []
        all_targets = []
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                outputs = model(X_batch)
                all_preds.append(outputs.cpu().numpy())
                all_targets.append(y_batch.cpu().numpy())
        
        all_preds = np.concatenate(all_preds, axis=0)
        all_targets = np.concatenate(all_targets, axis=0)
        
        # Calculate weighted F1 score for both tasks
        f1_score_avg = weighted_f1_score(all_preds, all_targets, all_targets[:, 0], all_targets[:, 1])
        
        print(f"Epoch {epoch+1}/{num_epochs} - Val Weighted F1 Score: {f1_score_avg:.4f}")
        
        # Store model weights and validation F1 score
        model_weights_per_epoch.append(model.state_dict())  # Save weights
        val_f1_scores.append(f1_score_avg)  # Save validation score

    # After training, find the epoch with the best validation F1 score
    best_epoch = np.argmax(val_f1_scores)  # Get the index of the best F1 score
    print(f"Best epoch: {best_epoch + 1} with F1 Score: {val_f1_scores[best_epoch]:.4f}")
    
    # Reload the best model weights
    model.load_state_dict(model_weights_per_epoch[best_epoch])  # Load best model weights
    return model




In [17]:

# --- Load the target data from a separate .xlsx file ---
target_file_path = 'widsdatathon2025/TRAIN/TRAINING_SOLUTIONS.xlsx' 
target_data = pd.read_excel(target_file_path)

# Merging the target columns with the training data
train_data_with_targets = pd.merge(train_data_imputed_df, target_data, on='participant_id', how='inner')


# Define target columns for ADHD and Sex
target_cols = ['ADHD_Outcome', 'Sex_F']

# Train the model
trained_model = train_model(train_data_with_targets, target_cols, num_epochs=100, batch_size=32, learning_rate=0.0001, weight_decay=0.01, dropout_rate=0.2)



Epoch 1/100 - Val Weighted F1 Score: 0.6737
Epoch 2/100 - Val Weighted F1 Score: 0.4316
Epoch 3/100 - Val Weighted F1 Score: 0.4863
Epoch 4/100 - Val Weighted F1 Score: 0.4316
Epoch 5/100 - Val Weighted F1 Score: 0.5224
Epoch 6/100 - Val Weighted F1 Score: 0.4461
Epoch 7/100 - Val Weighted F1 Score: 0.7669
Epoch 8/100 - Val Weighted F1 Score: 0.0000
Epoch 9/100 - Val Weighted F1 Score: 0.3893
Epoch 10/100 - Val Weighted F1 Score: 0.7694
Epoch 11/100 - Val Weighted F1 Score: 0.7362
Epoch 12/100 - Val Weighted F1 Score: 0.7728
Epoch 13/100 - Val Weighted F1 Score: 0.7596
Epoch 14/100 - Val Weighted F1 Score: 0.4316
Epoch 15/100 - Val Weighted F1 Score: 0.6120
Epoch 16/100 - Val Weighted F1 Score: 0.5614
Epoch 17/100 - Val Weighted F1 Score: 0.4365
Epoch 18/100 - Val Weighted F1 Score: 0.6440
Epoch 19/100 - Val Weighted F1 Score: 0.6903
Epoch 20/100 - Val Weighted F1 Score: 0.6838
Epoch 21/100 - Val Weighted F1 Score: 0.4072
Epoch 22/100 - Val Weighted F1 Score: 0.6289
Epoch 23/100 - Val 

#### Predicting ADHD and Sex

In [18]:
import pandas as pd
import torch

def predict_and_save_csv(trained_model, test_df, device, output_path='test_predictions.csv'):
    # Step 1: Preprocess the test data (same as training preprocessing)
    X_test = test_df.drop(columns=['participant_id'])  # Features only
    X_test = X_test.apply(pd.to_numeric, errors='coerce')  # Ensure all data is numeric
    
    # Handle any NaNs in test data by filling with the mean (or other strategy)
    X_test.fillna(X_test.mean(), inplace=True)

    # Convert to torch tensor and move to the appropriate device (CPU or GPU)
    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
    
    # Step 2: Make predictions
    trained_model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        outputs = trained_model(X_test_tensor)
        
        # Outputs will be [batch_size, 2], where outputs[:, 0] corresponds to Sex_F, and outputs[:, 1] corresponds to ADHD_Outcome
        predictions = (outputs.cpu().numpy() > 0.5).astype(int)  # Convert to 0 or 1 for both targets
    
    # Step 3: Prepare the output DataFrame with participant_id, ADHD_Outcome, and Sex_F
    results = pd.DataFrame(predictions, columns=['Sex_F', 'ADHD_Outcome'])
    
    # Ensure participant_id is a 1D array, not 2D
    results['participant_id'] = test_df['participant_id'].values  # Ensure participant_id is a 1D array
    
    # Step 4: Reorder the columns to match the desired format
    results = results[['participant_id', 'ADHD_Outcome', 'Sex_F']]
    
    # Step 5: Save the predictions to a CSV file
    results.to_csv(output_path, index=False)
    print(f"Predictions saved to {output_path}")


# Usage
# Assuming `trained_model` is your trained model and `test_data` is your test DataFrame
# And `device` is either 'cuda' (GPU) or 'cpu'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
predict_and_save_csv(trained_model, test_data_imputed_df, device, output_path='test_predictions.csv')



Predictions saved to test_predictions.csv


This is just me messing around with different models to see what happens. Ignore this for now

In [ ]:
# This doesn't work, something about keras that I don't understand 
# Also this code can only be ran one at a time because it's not picklable (new word of the day for me)
# Trying again with PyTorch - see next code block
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.base import BaseEstimator
from sklearn.metrics import make_scorer, accuracy_score

# Assuming you have already imputed the data (train_imputed_data, test_imputed_data, target_columns)

# Step 1: Separate features and targets for training
X_train = train_data_imputed_df.drop(columns=['participant_id'])  # Remove participant_id for training
y_train = train_data_with_targets[target_cols]  # Targets for training (ADHD_Outcome, Sex_F)

# Step 2: Separate features for test data
X_test = test_data_imputed_df.drop(columns=['participant_id'])
participant_id_test = test_data_imputed_df['participant_id']

# Step 3: Create the model function
def create_model(optimizer='adam', activation='relu', dropout_rate=0.2, weight_decay=1e-4):
    model = Sequential()
    model.add(Dense(128, input_dim=X_train.shape[1], activation=activation))  # Input layer
    model.add(Dropout(dropout_rate))  # Dropout layer
    model.add(Dense(64, activation=activation))  # Hidden layer
    model.add(Dropout(dropout_rate))  # Dropout layer
    model.add(Dense(2, activation='sigmoid'))  # Output layer (binary classification for both ADHD_Outcome and Sex_F)
    
    # Compile the model
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Custom estimator class to use TensorFlow with GridSearchCV
class KerasModel(BaseEstimator):
    def __init__(self, optimizer='adam', activation='relu', dropout_rate=0.2, weight_decay=1e-4):
        self.optimizer = optimizer
        self.activation = activation
        self.dropout_rate = dropout_rate
        self.weight_decay = weight_decay
        self.model = None

    def fit(self, X, y, batch_size=32, epochs=50):
        self.model = create_model(
            optimizer=self.optimizer,
            activation=self.activation,
            dropout_rate=self.dropout_rate,
            weight_decay=self.weight_decay
        )
        self.model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=0)
        return self

    def predict(self, X):
        return (self.model.predict(X) > 0.5).astype(int)  # Binary classification (0 or 1)

    def score(self, X, y):
        return self.model.evaluate(X, y, verbose=0)[1]  # Return accuracy

# Step 4: Create the model for GridSearchCV
model = KerasModel()

# Step 5: Define the hyperparameter grid for GridSearchCV
param_grid = {
    'optimizer': ['adam', 'sgd', 'rmsprop', 'adagrad'],  # Different optimizers
    'activation': ['relu', 'tanh', 'sigmoid', 'swish'],  # Different activation functions
    'dropout_rate': [0.1, 0.2, 0.3, 0.4, 0.5],  # Dropout rates for regularization
    'learning_rate': [1e-3, 1e-4, 1e-5, 1e-6],  # Learning rates
    'weight_decay': [0, 1e-4, 1e-3],  # Weight decay for L2 regularization
}

# Step 6: Create a custom scoring function
scorer = make_scorer(accuracy_score)

# Step 7: Use GridSearchCV to search the best hyperparameters with `n_jobs=1`
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3, verbose=2, scoring=scorer)

# Step 8: Fit the grid search on your training data
grid_result = grid.fit(X_train, y_train)

# Step 9: Get the best hyperparameters and print
best_params = grid_result.best_params_
print(f"Best Hyperparameters: {best_params}")

# Step 10: Create the final model using the best hyperparameters
final_model = KerasModel(
    optimizer=best_params['optimizer'],
    activation=best_params['activation'],
    dropout_rate=best_params['dropout_rate'],
    weight_decay=best_params['weight_decay']
)

# Step 11: Train the final model using the best hyperparameters
final_model.fit(X_train, y_train, batch_size=32, epochs=50)  # Default batch_size and epochs

# Step 12: Make predictions on the test set
predictions = final_model.predict(X_test)

# Step 13: Prepare the output DataFrame with participant_id, ADHD_Outcome, and Sex_F
results = pd.DataFrame(predictions, columns=['ADHD_Outcome', 'Sex_F'])
results['participant_id'] = participant_id_test.values

# Step 14: Reorder the columns
results = results[['participant_id', 'ADHD_Outcome', 'Sex_F']]

# Step 15: Save the results to a CSV file
results.to_csv('test_predictions_model2.csv', index=False)
print("Predictions saved to 'test_predictions_model2.csv'")

Fitting 3 folds for each of 960 candidates, totalling 2880 fits
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/52/v1gpfmxd54d6bqwl7msx6jb40000gn/T/ipykernel_72410/3241726433.py", line 79, in <module>
    grid_result = grid.fit(X_train, y_train)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py", line 1152, in wrapper
    This mixin is empty, and only exists to indicate that the estimator is a
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_search.py", line 898, in fit
    Training vectors, where `n_samples` is the number of samples and
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/

In [ ]:
# This is not NOT working, I just haven't finessed it in any way. It's a straight copy from chatgpt
# I am going to bed though so I will try again tomorrow to actually make it fit in with the data
# To fix it involves changing y_test into something that can be predicted
# Similar to what I have above with the keras model but, like, better :)
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
import pandas as pd

# Step 1: Load your preprocessed data (this step is assumed to be done)
# Assuming train_data_imputed_df and test_data_imputed_df are ready, along with target_cols
# Let's separate features and targets for training
X_train = train_data_imputed_df.drop(columns=['participant_id'])
y_train = target_cols  # Targets for training (ADHD_Outcome, Sex_F)

X_test = test_data_imputed_df.drop(columns=['participant_id'])
y_test = target_cols  # Assuming test data has the same target columns for evaluation

# Step 2: Define the neural network model
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_rate=0.2, activation='relu'):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, output_dim)
        self.dropout = nn.Dropout(p=dropout_rate)
        
        if activation == 'relu':
            self.activation = torch.relu
        elif activation == 'sigmoid':
            self.activation = torch.sigmoid
        elif activation == 'tanh':
            self.activation = torch.tanh
        else:
            self.activation = torch.relu  # default
        
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.dropout(x)
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        return torch.sigmoid(x)  # Sigmoid output for binary classification

# Step 3: Define the function to train and evaluate the model
def train_and_evaluate_model(X_train, y_train, X_test, y_test, 
                              optimizer_name='adam', activation='relu', 
                              dropout_rate=0.2, batch_size=32, epochs=50, learning_rate=1e-3):
    # Prepare data
    X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)
    
    # Initialize the model
    model = NeuralNetwork(input_dim=X_train.shape[1], output_dim=y_train.shape[1], 
                          dropout_rate=dropout_rate, activation=activation)
    
    # Define the loss function and optimizer
    criterion = nn.BCELoss()
    if optimizer_name == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    elif optimizer_name == 'rmsprop':
        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
    else:
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Default to Adam
    
    # Training loop
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()  # Update weights

    # Evaluation loop
    model.eval()
    with torch.no_grad():  # No need to calculate gradients for evaluation
        test_predictions = model(X_test_tensor)
        predicted_classes = (test_predictions > 0.5).float()  # Apply threshold
        accuracy = accuracy_score(y_test_tensor.numpy(), predicted_classes.numpy())
    
    return accuracy

# Step 4: Define hyperparameter search space
param_grid = {
    'optimizer': ['adam', 'sgd', 'rmsprop'],
    'activation': ['relu', 'sigmoid', 'tanh'],
    'dropout_rate': [0.1, 0.2, 0.3],
    'batch_size': [16, 32, 64],
    'epochs': [10, 30, 50],
    'learning_rate': [1e-3, 1e-4, 1e-5],
}

# Step 5: Loop through the hyperparameters and train the models
best_accuracy = 0
best_params = {}

for optimizer in param_grid['optimizer']:
    for activation in param_grid['activation']:
        for dropout_rate in param_grid['dropout_rate']:
            for batch_size in param_grid['batch_size']:
                for epochs in param_grid['epochs']:
                    for learning_rate in param_grid['learning_rate']:
                        # Train and evaluate the model with these hyperparameters
                        accuracy = train_and_evaluate_model(
                            X_train, y_train, X_test, y_test,
                            optimizer_name=optimizer, activation=activation, 
                            dropout_rate=dropout_rate, batch_size=batch_size, 
                            epochs=epochs, learning_rate=learning_rate
                        )
                        print(f"Tested {optimizer}, {activation}, {dropout_rate}, {batch_size}, {epochs}, {learning_rate} -> Accuracy: {accuracy}")
                        
                        # Track the best model
                        if accuracy > best_accuracy:
                            best_accuracy = accuracy
                            best_params = {
                                'optimizer': optimizer,
                                'activation': activation,
                                'dropout_rate': dropout_rate,
                                'batch_size': batch_size,
                                'epochs': epochs,
                                'learning_rate': learning_rate
                            }

# Step 6: Print best hyperparameters and accuracy
print("Best Hyperparameters:", best_params)
print("Best Accuracy:", best_accuracy)
